# 01 - Preparing your proteins

## Introduction

The prepare_proteins library was written to deal with the high throughput setup of protein systems. It can handle many PDB files simultaneously to set up general optimizations that prepare the systems for specific calculations and simulations.

In this document, we will show an example of the general workflow that can be followed to accomplish the previously mentioned objectives. We will work with several glutathione peroxidase (GPX) sequences from building their models (with Alpha Fold) to setting up PELE simulations.

## 1. What modules and libraries do we need?

First, we need to import the main library **"prepare_proteins"**. 

Second, we will also import an additional library to help us send calculations to the different BSC clusters. The **"bsc_calculations"** library sets up the calculation files, folders and slurm scripts for efficiently launching jobs to the clusters. 

In [ ]:
import prepare_proteins
import bsc_calculations

We will also load other common Python libraries to help us in out set up:

In [ ]:
import os
import shutil

## 2. Preparing sequences - starting from a FASTA file

In this case, we are starting from protein sequences, so we need to model their protein structures. We will set up AlphaFold calculations from a FASTA file ("gpx_sequences.fasta") containing five GPX sequences. 

The first step is to initialise the *sequenceModels* class with the path to our fasta file. We assigned the initialised class to the variable *sequences*:

In [ ]:
sequences = prepare_proteins.sequenceModels('gpx_sequences.fasta')

Now we can use the class method *setUpAlphafold* to create all the files, folders and commands to launch AlphaFold. It takes as the only parameter the folder's name in which we want to put our calculation files. The method returns a list of the commands that must be executed to run the job. We store that list in a variable called jobs:

In [ ]:
jobs = sequences.setUpAlphaFold('alphafold')

Finally, we can create a slurm script to launch the AlphaFold jobs using the **"bsc_calculations"** library. Since the job will be run in the Minotauro cluster, we call a method inside the corresponding sub-class from the library:

In [ ]:
bsc_calculations.minotauro.jobArrays(jobs, job_name='AF_sequences', partition='bsc_ls', program='alphafold')

The *jobArrays* method needs the list of commands to generate the slurm script file. We have specified the 'bsc_ls' partition to run the calculations, and with the keyword "program", we tell the script to load all necessary libraries to run AlphaFold in this cluster.

To launch the calculations you will need to upload the 'AF_sequences' folder and the 'slurm_array.sh' script to the cluster and then launch it with: 

    sbatch slurm_array.sh

After all the AlphaFold calculation has finished, we will need to get the protein structures output from the cluster. Since AlphaFold generates large-memory outputs we are only interested in grabbing the PDB files to load them into our library. This can be easily done with a command like this:

    tar cvf AF_sequences.tar AF_sequences/output_models/\*/relaxed_model_\*pdb"

The tar file contains only the relaxed PDB outputs but mantains the folder structure of our AlphaFold calculation.

### 2.2. Preparing models - taking PDB files

After we get our AlphaFold results from the cluster we need to put them into a folder renamed woth their corrsponfing protein names. To do that we run the following code:

In [ ]:
# Create a structures folder if it does not exists
if not os.path.exists('structures'):
    os.mkdir('structures')
    
# Copy each alphafold output model (from a specific rank) into the structures folder
rank = 0
for model in os.listdir('alphafold/output_models/'):
    if os.path.exists('alphafold/output_models/'+model+'/ranked_'+str(rank)+'.pdb'):
        shutil.copyfile('alphafold/output_models/'+model+'/ranked_'+str(rank)+'.pdb', 
                        'structures/'+model+'.pdb')

Now we can initialise the *proteinModels* class with our PDB files from the structures folder:

In [ ]:
models = prepare_proteins.proteinModels('structures')

The library reads all PDB files as [biopython Bio.PDB.Structure()](https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ) objects at the structures attribute. The attribute is a dictionary whose keys are the protein models names and the values are the Bio.PDB objects. The library can be iterated to get the protein models names at each iteration:

In [ ]:
for model in models:
    print(models.structures[model])

## 3. System preparation

### 3.1 Removing low confidence regions from AlphaFold models at the protein termini

AlphaFold models can contain structural regions with low confidence in their prediction. Since this can represent large structural domains or segments, we are interested in removing them, specially if their are found at the N- and C-termini.

The **prepapare_proteins** library has a method to remove terminal segments from AlphaFold structures using the confidence score stored at the B-factor column of the PDBs:

In [ ]:
models.removeTerminiByConfidenceScore(confidence_threshold=90)

The condifedence_threshold keyword indicates the maximum confidence score at which to stop the triming of terminal regions. 

At any point, when we are modifying our proteins it is a good idea to check that the structural changes have been carried out as we expected. The library has a method for writing all the structures into a folder so we can visualise the state of our set up:

In [ ]:
models.saveModels('trimed_models')

We can open the PDB files with any external programs to check what the previous code did.

In the current state of the library, after some modifications on the structures, we need to re-initialise the *proteinModels* class using the models written to a folder with the saveModels() method. For this we simply repeat:

In [ ]:
models = prepare_proteins.proteinModels('trimed_models')

### 3.2 Align structures to a reference PDB

When comparing related proteins is a good idea to align them to have a common structural framework. The library helps you align the proteins with the method alignModelsToReferencePDB(). We need to give a reference PDB (which, as in this case, can be any PDB from our models), then a folder where to write the aligned structures, and the index (or indedxes) of the chains to align (see the documentation inside the function for details on how the chain_indexes are given). For now we set up the index to be the first folder in the structure (chain_indexes=0) and we run the alignement:

In [ ]:
models.alignModelsToReferencePDB('trimed_models/GPX_Bacillus-subtilis.pdb', 'aligned_models', chain_indexes=0)

We will continue working with the aligned structures, for that we reload our output models into the library:

In [ ]:
models = prepare_proteins.proteinModels('aligned_models/')

# 4. Prepwizard optimizations

After our protein models are correctly trimed and aligned we can continue with the Prepwizard optimaztion of the structures. We create this set up by calling the method setUpPrepwizardOptimization():

In [ ]:
jobs = models.setUpPrepwizardOptimization('prepwizard')

Again, the method needs a folder name to put all inout files for the calculations. After executing the method it returns the comamnds to be executed for running the Prepwizard optimization in a machine endowed with the Schrodinger Software license. The commands can be passed to the **bsc_calculations** library to generate the scripts for facilitate the execution:

In [ ]:
bsc_calculations.local.parallel(jobs, cpus=min([40, len(jobs)]))

We pass the commands to the local sub-class and define the number of cpus we want to use beforehand, so the library will create one script for each CPU to be used. In our case, we are working with 5 files, so only 5 script files were created. The script files are named commands_?, where ? stands for an integer identifying the individual script. The method also writes a script named commands, which can launch all the scripts simultaneously. So, in order to run the optimizations, we need tu upload the 'prepwizard' folder to a cluster with Schrodinger license together with all the commands scripts. Then to launch the calculation you need to run:

    bash commands
    
The execution can be followed by looking at the log files being generated at the output_models folder inside the 'prepwizard' job folder. 

After all the Prepwizard optimization jobs are over we need to donwload them into the folder containing this notebook and load them into the library with:

In [ ]:
models.loadModelsFromPrepwizardFolder('prepwizard/')

If any model is missing from the optimization the library will issue a warning specifying which models were not found. If that's the case we suggest to look at the log files to check what could have been wrong. 

After loading the optimization models into the library, it is again recommended to check them visually. For that we save them into a 'prepwizard_models' folder:

In [ ]:
models.saveModels('prepwizard_models')

# 5. Grid set up for Glide docking

In order to run PELE we need to generate starting ligand conformations for our systems. We can achieve this by running docking simulations with Glide docking. The first step to achieve this is to generate grid files suitable for protein ligand docking calculations using the Glide docking engine provided by Schrodinger. 

## 5.1 Defining residues for the center of the docking calculations

To set the grid calculations we first need to define the atoms serving as the center for our docking calculations. When working with multiple proteins this can be complicated since the residues have different PDB indexes. To facilitate the identification of equivalent residues in an evolutionary framework of related proteins we employ multiple sequence alignments (MSAs) of all the loaded protein models. The library can easiliy run a MSA calculation using the mafft (command-line) program. The program is called internally, so we only need to execute:

In [ ]:
msa = models.calculateMSA()

The 'msa' object is read into the framework of [Biopython's Bio.Align.MultipleSeqAlignment](https://biopython-tutorial.readthedocs.io/en/latest/notebooks/06%20-%20Multiple%20Sequence%20Alignment%20objects.html) object. We can write this multiple sequence alignment to a file to check it with an external program (we recommend using chimera, because it can easily map the sequence to the structures, but any would do for a general purpose):

In [ ]:
prepare_proteins.alignment.writeMsaToFastaFile(msa, 'msa.fasta')

In our case we are interested in a specific cysteine residue conserved in all the sequences. To speed up the search of the msa position corresponding to this conserved residue we can use a method to find all conserved MSA positions and their identies. Since this tutorial uses very few protein models there are several conserved residues, however when a high number of proteins are compared, the conservation become more meaningful. For our case we will print only the Cysteine residues:

In [ ]:
conserved_index = models.getConservedMSAPositions(msa)
for c in conserved_index:
    if c[1] == 'C':
        print(c)    

Two conserved residues are present, a quick search of the literature defines the conserved Cysteine residue as the one with MSA index 33. We employ this MSA index to get the PDB indexes for all the proteins loaded into our library:

In [ ]:
cys_index = models.getStructurePositionFromMSAindex(33)
print(cys_index)

With those indexes we can continue to define the center of our grid calculation.

## 5.2 Set up the grid calculation

We define the center of the grid as the 'SG' atom of the Cysteine residues previusly defined. For this we build a dictionary defining the atoms as 3-elements tuples: (chain_id, residue_id, atom_name). To grab all this information is safest if we employ the Bio.PDB.Structure objects inside the library. We are going to iterate these objects searching for the residues that match our indexes:

In [ ]:
cys_center_atom = {} # Create dictionary to store the atom 3-element tuple for each model
for model in models: # Iterate the models inside the library
    for r in models.structures[model].get_residues(): # Iterate the residues for each Bio.PDB.Structure object
        if r.id[1] == cys_index[model]: # Check that the residue matches the defined index
            assert r.resname == 'CYS' # Assert that the residue has the correct residue identity
            cys_center_atom[model] = (r.get_parent().id, r.id[1], 'SG') # Store the corresponsing tuple.

Now that we have our center atoms we proceede to set up the grid calculations folder and script, analogously to what we did with the Prepwizard optimization step:

In [ ]:
jobs = models.setUpDockingGrid('grid', cys_center_atom) # Set grid calcualtion
bsc_calculations.local.parallel(jobs, cpus=min([40, len(jobs)])) # Write the scripts to run them locally.

The setUpDockingGrid() method needs the folder name where to put the calculation's files and folders, but also, as a second argument, the dictionary with the center atoms (one per each model in the library).

We also create the scripts for running the grid calcualtion (note that this will overwrite the previous scripts written down for the Prepwizard calculations, so it is always a good idea to check these scripts before running them).

The launching of the calculation is very similar to the Prepwizard optimization case, so you can check above for details.

After the calculation has finished, we need to download the results, however is recommended not to delete the outputs from the cluster where they were run; we still will need them for the Glide Docking job.

# 6. Set up Glide docking

Running Glide Docking in our proteins will obviously need a set of ligands to be docked. It is an usual practice to draw them in the free Maestro 2D sketcher and store them as .mae files into a common folder. However, in this tutorial, we have donwloaded the ligand directly from the PDB database and therefore it will need to be converted to the .mae format. This task can become tediously manual if the number of ligands is very large, this is the reason why we have implemented a method for converting all ligand PDB files inside a specific folder into the .mae format:

In [ ]:
models.convertLigandPDBtoMae('ligands')

After running the function, the folder will contain the .mae files inside it, and now can be given to the docking set up method:

In [ ]:
jobs = models.setUpGlideDocking('docking', 'grid', 'ligands', poses_per_lig=100)
bsc_calculations.local.parallel(jobs, cpus=min([40, len(jobs)]))

The setUpGlideDocking() function needs three mandatory arguments, the 'docking' folder, where the docking job will be stored; the 'grid' folder, where the grid calculation outputs are located; and the 'ligands', folder with the ligand structures in .mae format. We have specified the number of docking trajectories to be run with the keyword poses_per_lig.

After running and downloading the docking results they can be analysed to extract the best poses to feed them to the PELE set up.

# 7. Analye docking calculations

## 7.1 Calculating docking distances

The best docking structures should be selected according to their Glide Score, however, in many ocassions, we also would like to filter the poses by different protein-ligand distances that represent the best conformations according to what we are aiming to simulate. For this reason we are going to do the docking analysis calculating the distance between the ligand sulfur atom and the SG atom of the catalytic cysteine of the GPX enzymes.

First, we build a dictionary containing the docking distances we want to calculate from the conformations genereated from the Glide docking calculation. This dictionary (atom_pairs) is a doubly nested dictionary that should go first for all protein models and then for all the ligands that were docked:

atom_pairs = {\
$\;\;\;\;$model1 : {\
$\;\;\;\;\;\;\;\;$ligand1 : [(protin_atom1, ligand_atom1), (protin_atom1, ligand_atom1), etc...],\
$\;\;\;\;\;\;\;\;$ligand2 : [(protin_atom1, ligand_atom1), (protin_atom1, ligand_atom1), etc...]},\
$\;\;\;\;$model2 : {\
$\;\;\;\;\;\;\;\;$ligand1 : [(protin_atom1, ligand_atom1), (protin_atom1, ligand_atom1), etc...],\
$\;\;\;\;\;\;\;\;$ligand2 : [(protin_atom1, ligand_atom1), (protin_atom1, ligand_atom1), etc...]},\
$\;\;\;\;$etc...\
}

Each doubly nested dictionary, representing the analysis for each protein and ligand, should contain a list of the atom pairs to use in the distance calculation. Similarly as was done for the atom_centers to define the grid of the docking, each atom must be represented as 3-element tuple (see above) or, in the case of the ligand atom, just with a string representing the ligand atom name.

We build the dictionary by iterating the models and by combining the ligand sulphur atom name (S1) with Cysteine residues previously used as atom centers of the grid calculation:

In [ ]:
atom_pairs = {} # Define the dictionary containing the atom pairs for each model
for model in models:
    atom_pairs[model] = {} 
    for ligand in ['GSH']:
        atom_pairs[model][ligand] = []
        atom_pairs[model][ligand].append((cys_center_atom[model], 'S1'))
atom_pairs

We can now use this nested dictionary as input for our docking analysis:

In [ ]:
models.analyseDocking('docking', atom_pairs=atom_pairs)

The docking analysis data is stored as a [panda dataframe](https://pandas.pydata.org/) in the attribute .docking_data:

In [ ]:
models.docking_data

## 7.2 Selecting the best poses according to a common metric

To facilitate pose selection according to a common distance we first need to group the distances under a common name (from now on metric) in our dataframe. For this, we can use the method combineDockingDistancesIntoMetrics(), which will gather all distances in a list and combine them by taking the minimum of the distance values. To use the function, we (again) need to construct a triply nested dictionary which goes from metric, model, and ligand, and contains as values the lists (of each model + ligand) which will be combined under the common metric name:

metric_distances = {\
$\;\;\;\;$metric_label_1 : {\
$\;\;\;\;\;\;\;\;$model1 : {\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand1 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand2 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;$model2 : {\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand1 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand2 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;$etc...\
$\;\;\;\;$metric_label_2 : {\
$\;\;\;\;\;\;\;\;$model1 : {\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand1 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand2 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;$model2 : {\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand1 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;\;\;\;\;$ligand2 : [(distance_label_1, distance_label_2, etc...],\
$\;\;\;\;\;\;\;\;$etc...\
$\;\;\;\;$etc...\
}

The construction of this dictionary is facilitated by employing the method getDockingDistances(), which will return the  list of distance labels associated with a specific protein + ligand docking. Since all distances calculated in this example will be stored under the same metric we can straigthforwardly build the 'metric_distances' dicionary:

In [ ]:
metric_distances = {} # Define the global dictionary
metric_distances['SG_S'] = {} # Define the metric nested dictionary
for model in models:
    metric_distances['SG_S'][model] = {} # Define the model nested dictionary
    for ligand in models.docking_ligands[model]: 
        # Define the ligand nested dictionary with all the docking distances list
        metric_distances['SG_S'][model][ligand] = models.getDockingDistances(model, ligand)

We now give this dicionary to the combineDockingDistancesIntoMetrics() method and inspect the changes upon our docking data:

In [ ]:
models.combineDockingDistancesIntoMetrics(metric_distances)
models.docking_data

Now that our docking data dataframe contains a new column with the docking distances grouped under the same name, we can use this information to extract the best poses resulting from each docking run. 

This can be achieved by giving a threshold value to obtain all catalytic poses fufilling a specific value for the common metric and getting the ones with the best Glide score, however, it is not always the case that all docking runs (i.e., for each protein and ligand combination) produce good distances, so many docking results would be left out if the distance thresholds employed are too restrictive. A way to solvent this would be to select a small distance threshold and select the best models that fulfilled it and then increase this threshold in a small amount to gather the best models that contain slightly worse metric distances. If we repeat this iteratively we could get the best possible models for all the docking runs without compromising using a large threshold for poses with good metric values. 

Luckily, this process has been automated in the library so the docking selection can be carried out easily:

In [ ]:
best_poses = models.getBestDockingPosesIteratively(metric_distances, max_threshold=7)
best_poses

To use the function we only need to give the name of the metrics that will be employed in the filtering.* The fucntion defines the iteration process with the following keywords: 

min_threshold=3.5\
max_threshold=5.0\
step_size=0.1

This is similar as the [np.arange()](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) function works and it can be tweaked to obtain optimal results for the selection you are aiming to get. 

*(Caution: in the current implementation of the function the same threshold value is employed at each iteration for all metric values). 

The best poses thus selected are returned as a pandas dataframe and are used to set up the final PELE calculation.

## 7.3 Extracting docking poses

We can extract a subset of docking poses by just giving a filtered pandas dataframe to the method extractDockingPoses(). 
In our case we use the 'best_poses' dataframe containing one pose per docking run.

In [ ]:
models.extractDockingPoses(best_poses, 'docking', 'best_poses', separator='@')

The inputs are the filtered dataframe, the 'docking' folder, and the folder where to store the poses. The name of the files will be given as a combination between the model_name+ligand+docking_pose and will be separated by the symbol given at the kwyeord separator. If the protein or ligand name contains the separator in their name a warning will be rised.

# 8. Setting up PELE

Once the docking poses are extracted, the final step is setting up the PELE calculation folders, files, and slurm scripts. The method needs as inputs the folder where to store the calcualtion information ('pele'), the folder contaning the docking poses ('best_poses'), and an input yaml file containing the details of the PELE platform protocol. We also give our atom_pairs dictionary to calculate the same distances employed for our docking analysis, so they are calcualted throughout the PELE simulation. 

Since the ligand index in our docking poses is zero we change the default ligand index (1) with the keyword  ligand_index. Finally, since we change the separator when writing the docking poses, we need to make aware the setUpPELECalculation() method by using the separator keyword.

In [ ]:
jobs = models.setUpPELECalculation('pele', 'best_poses/', 'input.yaml', 
                                   distances=atom_pairs, ligand_index=0, separator='@')

The commands generated by this method are slightly different than in other functions. However, they can be easily processed by employing the setUpPELEForMarenostrum() method inside the bsc_calculations library. 

In [ ]:
bsc_calculations.marenostrum.setUpPELEForMarenostrum(jobs)